In [1]:
import inro.modeller as m
import os
import glob
mm = m.Modeller()
eb = mm.emmebank
import inro.emme.matrix as _matrix
import pandas as pd
import numpy as np
from copy import deepcopy

matrix_calculator = m.Modeller().tool("inro.emme.matrix_calculation.matrix_calculator")
create_matrix =  m.Modeller().tool("inro.emme.data.matrix.create_matrix")
change_matrix = m.Modeller().tool("inro.emme.data.matrix.change_matrix_properties") 
import_matrices = mm.tool("tmg.input_output.import_binary_matrix")  
matrix_transaction = mm.tool('inro.emme.data.matrix.matrix_transaction')
create_extra = m.Modeller().tool("inro.emme.data.extra_attribute.create_extra_attribute")
network_calc= m.Modeller().tool("inro.emme.network_calculation.network_calculator")

def clear_matrices(emmebank, exception_array = [""], type = "FULL"):
    for matrix in emmebank.matrices():
        if matrix.type == type:
            if matrix.id not in exception_array:
                emmebank.delete_matrix(matrix.id)

def import_matrices_from_directory(matrix_folder, matrix_list, extension, scenario):
    #imports all matrices in directory and subfolders of the specified extension type
    #matrix_list should be in the format:
    #   {file_name1: matrix_num1, file_name2: matrix_num2, ...}
    import os

    for root, dirs, files in os.walk(matrix_folder):
        for matrix_file in files:
            if matrix_file.endswith(extension):
                name = os.path.splitext(matrix_file)[0]
                if name in matrix_list:
                    matrix_number = int(matrix_list[name][2:])
                    import_matrices(4, matrix_number ,os.path.join(root, matrix_file),scenario, name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_") )
                    print "Imported matrix " + matrix_list[name]
                    change_matrix(matrix = matrix_list[name],
                        matrix_name = name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_")[:40],
                        matrix_description = name)
    
    #check
    for name in matrix_list:
        if eb.matrix(matrix_list[name]): 
            n_matrix = eb.matrix(matrix_list[name]).get_numpy_data(scenario)
            if n_matrix.sum() == 0 :
                print "Matrix %s was not imported, or contains no values!" % name
        else:
            print "Matrix %s was not imported!" % name
    print "Finished Importing Matrices"
    
#obtain the minimum transit travel time between all modes.
#uses outputs from HDR transit time toolbox
mcSpec = {
    "expression": None,
    "result": "",
    "constraint": {
        "by_value": None,
        "by_zone": None
    },
    "aggregation": {
        "origins": None,
        "destinations": None
    },
    "type": "MATRIX_CALCULATION"
}
def minimum_transit_time(tt_dir, min_transit_time,scenario,go_sub_mtx_nums):
    
    #import the .311 files containing GO Train and subway OD travel times
    for root, dirs, files in os.walk(tt_dir):
        for matrix_file in files:
            if matrix_file.endswith(".311"):
                matrix_transaction(os.path.join(root, matrix_file),scenario)
                
    create_matrix(matrix_id = min_transit_time,
        matrix_name= "min.transit.time",
        matrix_description = "minimum.transit.time")

    matrix_list ={ "express.bus.actual": "",
                   "rapid.bus.actual":"",
                   "local.bus.actual":""
                   }

    #Import matrices
    for matrix_name in matrix_list.keys():
        matrix_list[matrix_name] = eb.available_matrix_identifier('FULL')
        create_matrix(matrix_id = matrix_list[matrix_name],
                      matrix_name= matrix_name,
                      matrix_description = matrix_name)

    import_matrices_from_directory(tt_dir, matrix_list, ".mdf", scenario)
    
    spec = {
           "type": "MATRIX_CALCULATION",
           "result": min_transit_time,
           "expression":  "(" + matrix_list["express.bus.actual"] + ".min."\
            + matrix_list["rapid.bus.actual"] + ".min."\
            + matrix_list["local.bus.actual"] + ")"
    }
    matrix_calculator(spec)

    for mtx_num in go_sub_mtx_nums:
        expression = min_transit_time + ".min.mf" + mtx_num + "*(mf" + mtx_num + "!=0)+" + min_transit_time + "*(mf" + mtx_num + "==0)"
        spec = {
           "type": "MATRIX_CALCULATION",
           "result": min_transit_time,
           "expression":  expression
        }
        matrix_calculator(spec)

    print "Finished Calculating OD Transit Travel Times"
    
def minimum_transit_time_mto(tt_dir,mto_skims, min_transit_time, scenario):            
    create_matrix(matrix_id = min_transit_time,
        matrix_name= "min.transit.time",
        matrix_description = "minimum.transit.time")

    matrix_list ={"express.bus.actual": "",
                   "rapid.bus.actual":"",
                   "local.bus.actual":""
                   }
    
    mat_list_mto = {"Express Bus": "",
                   "Local Bus": "",
                   "Rapid Bus": "",
                   "Subway": "",
                   "Train": ""}

    matching = {"Express Bus": "express.bus.actual",
               "Local Bus":"local.bus.actual",
               "Rapid Bus":"rapid.bus.actual"}
    #Import matrices
    for matrix_name in matrix_list.keys():
        matrix_list[matrix_name] = eb.available_matrix_identifier('FULL')
        create_matrix(matrix_id = matrix_list[matrix_name],
                      matrix_name= matrix_name,
                      matrix_description = matrix_name)

    import_matrices_from_directory(tt_dir, matrix_list, ".mdf", scenario)
    
    for matrix_name in mat_list_mto.keys():
        mat_list_mto[matrix_name] = eb.available_matrix_identifier('FULL')
        create_matrix(matrix_id = mat_list_mto[matrix_name],
                  matrix_name= matrix_name.replace(" ","_"),
                  matrix_description = matrix_name)

    import_matrices_from_directory(mto_skims, mat_list_mto, ".mdf", scenario)
    
    for match in matching.keys():
        mto = mat_list_mto[match]
        hdr = matrix_list[matching[match]]
        spec = {
           "type": "MATRIX_CALCULATION",
           "result": mto,
           "expression":  hdr,
             "constraint": {
        "by_value": {
            "interval_min": 0,
            "interval_max": 0,
            "condition": "INCLUDE",
            "od_values": mto
        },
        "by_zone": None}
            }
        matrix_calculator(spec)
    min_exp = "("
    for matrix_name in mat_list_mto.keys():
        mat_id = mat_list_mto[matrix_name]
        min_exp = min_exp + mat_id + ".min."
        spec = {
           "type": "MATRIX_CALCULATION",
           "result": mat_id,
           "expression":  "9999",
             "constraint": {
        "by_value": {
            "interval_min": 0,
            "interval_max": 0,
            "condition": "INCLUDE",
            "od_values": mat_id
        },
        "by_zone": None}
            }
        matrix_calculator(spec)
    min_exp = min_exp[0:-5] + ")"

    spec = {
           "type": "MATRIX_CALCULATION",
           "result": min_transit_time,
           "expression":min_exp}
    matrix_calculator(spec)        
    spec = {
           "type": "MATRIX_CALCULATION",
           "result": min_transit_time,
           "expression":  "0",
        "constraint": {
        "by_value": {
            "interval_min": 9999,
            "interval_max": 9999,
            "condition": "INCLUDE",
            "od_values": min_transit_time
        },
        "by_zone": None}
            }
    matrix_calculator(spec)
    print "Finished Calculating OD Transit Travel Times"

def get_mc_spec(result, expression, agg_o, agg_d, con_o, con_d, con_val):
    specification = {
        "type": "MATRIX_CALCULATION",
        "result": result,
        "expression":  expression,
        "aggregation": {
            "origins": agg_o,
            "destinations": agg_d
        },
        "constraint": {
            "by_value": con_val,
            "by_zone": {
                "origins": con_o,
                "destinations": con_d
            }
        }
    }
    return specification

def MOE_fraction(out_file,numerator,denominator,zones_array):
    
    for origins in zones_array:
        spec = get_mc_spec(None, denominator,"+","+",origins,"all",None)
        bottom = matrix_calculator(spec)['result']
        spec = get_mc_spec(None, numerator,"+","+",origins,"all",None)
        top = float(matrix_calculator(spec)['result'])
        out_file.write("{0:.4f},".format(top/bottom))
    out_file.write("\n")
netCalcSpec = {
    "result": None,
    "expression": None,
    "aggregation": None,
    "selections": {
        "link": None
    },
    "type": "NETWORK_CALCULATION"
}

modes = {"sov":['trips_peak_hbw_auto1_nt', 'trips_peak_hbw_auto1_toll'],
        "hov2": ['trips_peak_hbw_auto2_nt_hov', 'trips_peak_hbw_auto2_nt_nh',
       'trips_peak_hbw_auto2_toll_hov', 'trips_peak_hbw_auto2_toll_nh'],
        "hov3": ['trips_peak_hbw_auto3_nt_hov', 'trips_peak_hbw_auto3_nt_nh',
       'trips_peak_hbw_auto3_toll_hov', 'trips_peak_hbw_auto3_toll_nh'],
        "Express Bus": ['trips_peak_hbw_transit_express_bus_walk','od_trips_peak_hbw_transit_express_bus_drive'],
        "Rapid Bus": ['trips_peak_hbw_transit_rapid_bus_walk','trips_peak_hbw_transit_rapid_bus_drive'],
        "Local Bus": ['trips_peak_hbw_transit_local_bus_walk','trips_peak_hbw_transit_local_bus_drive'],
        "Train":['od_trips_peak_hbw_transit_train_walk_access',
       'od_trips_peak_hbw_transit_train_transit_access',
       'od_trips_peak_hbw_transit_train_park_access',
       'od_trips_peak_hbw_transit_train_drop_off'],
        "Subway": ['od_trips_peak_hbw_transit_subway_walk_access',
       'od_trips_peak_hbw_transit_subway_transit_access',
       'od_trips_peak_hbw_transit_subway_park_access',
       'od_trips_peak_hbw_transit_subway_drop_off'],
        "walk": ['trips_peak_hbw_active_walk'],
        "bike": ['trips_peak_hbw_active_bike']}

In [2]:
#scenario folder names

#BAU
sc_2051_BAU = "2051\\Medium List\\BAU\\BAU2051_Updated_GGH_MTP_V106A3_CS222_20200703"
sc_2051_BAU_ogs = "2051\\Short List\\BAU\\BAU_ShortList_APTG_2051_GGH_MTP_V106A3_CS222_20201127"

#Network 1
sc_2051_N1_aptg = "2051\\Short List\\Network 1\\N1_ShortList_APTG_2051_GGH_MTP_V106A3_CS222_20201217"
sc_2051_N1_luv1 = "2051\\Medium List\\Network 1\\N1_LUV1_2051_GGH_MTP_V106A3_CS222_20200918"

#Network 2
sc_2051_N2_aptg = "2051\\Short List\\Network 2\\N2_ShortList_APTG_2051_GGH_MTP_V106A3_CS222_20201217"
sc_2051_N2_luv1 = "2051\\Medium List\\Network 2\\N2_LUV1_2051_GGH_MTP_V106A3_CS222_20200924"

#Network 3
sc_2051_N3_aptg = "2051\\Medium List\\Network 3\\N3_APTG_2051_GGH_MTP_V106A3_CS222_20200901"
sc_2051_N3_luv2 = "2051\\Medium List\\Network 3\\N3_LUV2_2051_GGH_MTP_V106A3_CS222_20201009"

#Network 4
sc_2051_N4_aptg = "2051\\Medium List\\Network 4\\N4_APTG_2051_GGH_MTP_V106A3_CS222_20200911"
sc_2051_N4_luv3 = "2051\\Medium List\\Network 4\\N4_LUV3_2051_GGH_MTP_V106A3_CS222_20201002"

#Network 5
sc_2051_N5_aptg = "2051\\Medium List\\Network 5\\N5_APTG_2051_GGH_MTP_V106A3_HS022_20200918"
sc_2051_N5_luv3 = "2051\\Medium List\\Network 5\\N5_LUV3_2051_GGH_MTP_V106A3_CS222_20201002"

## zone grouping definitions ## 
zones_pd1 = "gf1"
zones_airports = "ga1"
zones_ugc = "gb4"
zones_inner = "gc1"
zones_outer = "gc2"
zones_university = "gd1"
zones_employment = "ge1"
zones_hospitals = "gh1"
shed_central = "gs10;gs11"
rural_central = "gs11"
shed_east = "gs20;gs21"
rural_east = "gs21"
shed_midwest = "gs30;gs31"
rural_midwest = "gs31"
shed_southwest = "gs40;gs41"
rural_southwest = "gs41"
shed_west = "gs50;gs51"
rural_west = "gs51"
shed_north = "gs60;gs61"
rural_north = "gs61"
shed_niagara = "gs70;gs71"
rural_niagara = "gs71"
shed_urban = "gs10;gs20;gs30;gs40;gs50;gs60;gs70"
shed_rural = "gs11;gs21;gs31;gs41;gs51;gs61;gs71"
urban_lu = "gb2;gb3;gb4"

#auto occupancy
sov_oc = 1.2
hov2_oc = 2.2
hov3_oc = 3.3

#number of hours for transit assignment
num_hrs_am = 2
#num_hrs_md = 6

#Road costs and transit costs are half of what was provided by MTO in their estimates
#This is because I tag both sides of the link - and do not want to double count the cost of each project

roadCosts = {1: [5000000,4500000], # 2 lane widening
            2: [10000000,9000000], # 4 lane widening
            3: [15000000,13500000], # 6 lane widening
            4: [8375000,5750000], # 2 lane new construction
            5: [11875000,9250000], # 4 lane new construction
            6: [15375000,12750000], # 6 lane new construction
            7: [500000000,500000000], # 401 tunnel/overpass cost - Assumed it would be triple the cost of a normal 8 lane new construction
            8: [2500000,2500000], # Core/collector conversion cost
            9: [18875000,16250000]} # 8 lane new construction

transitCosts = {1: 12500000, # GO Train New Corridor (freight track only)
            2: 12500000, # GO Train New Corridor (no existing track)
            3: 5000000, # Go Train all day service and 15 minute
            4: 250000000, # Subway
            5: 62500000, # LRT
            6: 12000000, # BRT Lite
            8: 3500000, # GO train all day service
            9: 1500000, # GO train 15 minute service
            10: 5750000, #Ferry
            13: 25000000}  #BRT Dedicated Facility

bridgeCost = {1: [29000000,29000000],# Assumes 10000 m2 bridge, using prestressed concrete, as these will be wide highway bridges
             2: [205000000,190000000], # 4 leg interchange
             3: [80000000,75000000], # 3 leg interchange
             4: [100000000,30000000], # GO Station
             5: [5000000000,5000000000], #Missing Link cost
             6: [2500000000,2500000000], # Line 1/2 Capacity expansion surplus cost
             7: [100000000,100000000],#Kitchener Pearson realignment surplus cost
             8: [500000000,50000000]}  #Expensive Skyway Bridge
#Based on parametric estimating guide. This is a surplus cost - not per km of bridge

In [3]:
pop_o_matrix_ogs = "mo19"
emp_o_matrix_ogs = "mo20"
senior_o_matrix_ogs = "mo21"
low_inc_o_matrix_ogs = "mo22"# number of low inc households
youth_o_matrix_ogs = "mo23"
pop_d_matrix_ogs = "md92"
emp_d_matrix_ogs = "md93"
low_income_zones_ogs = "mo24"

pop_o_matrix_luv1 = "mo25"
emp_o_matrix_luv1 = "mo26"
senior_o_matrix_luv1 = "mo27"
low_inc_o_matrix_luv1 = "mo28"# number of low inc households
youth_o_matrix_luv1 = "mo29"
pop_d_matrix_luv1 = "md94"
emp_d_matrix_luv1 = "md95"
low_income_zones_luv1 = "mo30"

pop_o_matrix_luv2 = "mo31"
emp_o_matrix_luv2 = "mo32"
senior_o_matrix_luv2 = "mo33"
low_inc_o_matrix_luv2 = "mo34"# number of low inc households
youth_o_matrix_luv2 = "mo35"
pop_d_matrix_luv2 = "md96"
emp_d_matrix_luv2 = "md97"
low_income_zones_luv2 = "mo36"

pop_o_matrix_luv3 = "mo37"
emp_o_matrix_luv3 = "mo38"
senior_o_matrix_luv3 = "mo39"
low_inc_o_matrix_luv3 = "mo40"# number of low inc households
youth_o_matrix_luv3 = "mo41"
pop_d_matrix_luv3 = "md98"
emp_d_matrix_luv3 = "md99"
low_income_zones_luv3 = "mo42"

hospital_d = "md90"
postsec_d = "md91"
parking_d = "md24"

truck_folder = r"\\tore-infs01\PWExternal\451010\281829_GGH\GGHM Runs\2051\Long List\BAU\MTO_2051_GGH_MTP_5002_V106A3_HS032_20200117\Assignable matrices"
tt_dir = r"\\tore-infs01\Models\10039241 - GGH Multimodal Transportation\HDR_transit_time_results\20200212_2051_BAU_HS - Peak"
tt_dir_op = r"\\tore-infs01\Models\10039241 - GGH Multimodal Transportation\HDR_transit_time_results\20200212_2051_BAU_HS - Off-Peak"
hdr_bau_peak = "mf160"
hdr_bau_offpeak = "mf170"
local_fare_mat = "mf180"

In [4]:
#All runs

scens = {"2051_BAU":[sc_2051_BAU,[401,402,403],[pop_o_matrix_ogs,emp_o_matrix_ogs,senior_o_matrix_ogs,
                                                low_inc_o_matrix_ogs,youth_o_matrix_ogs,pop_d_matrix_ogs,
                                                emp_d_matrix_ogs,low_income_zones_ogs]],
        "2051_BAU_OGS":[sc_2051_BAU_ogs,[404,405,406],[pop_o_matrix_ogs,emp_o_matrix_ogs,senior_o_matrix_ogs,
                                                low_inc_o_matrix_ogs,youth_o_matrix_ogs,pop_d_matrix_ogs,
                                                emp_d_matrix_ogs,low_income_zones_ogs]],
         "2051_N1_APTG":[sc_2051_N1_aptg,[501,502,503],[pop_o_matrix_ogs,emp_o_matrix_ogs,senior_o_matrix_ogs,
                                                low_inc_o_matrix_ogs,youth_o_matrix_ogs,pop_d_matrix_ogs,
                                                emp_d_matrix_ogs,low_income_zones_ogs]],
         "2051_N1_LUV1":[sc_2051_N1_luv1,[504,505,506],[pop_o_matrix_luv1,emp_o_matrix_luv1,senior_o_matrix_luv1,
                                                low_inc_o_matrix_luv1,youth_o_matrix_luv1,pop_d_matrix_luv1,
                                                emp_d_matrix_luv1,low_income_zones_luv1]],
        "2051_N2_APTG":[sc_2051_N2_aptg,[601,602,603],[pop_o_matrix_ogs,emp_o_matrix_ogs,senior_o_matrix_ogs,
                                                low_inc_o_matrix_ogs,youth_o_matrix_ogs,pop_d_matrix_ogs,
                                                emp_d_matrix_ogs,low_income_zones_ogs]],
         "2051_N2_LUV1":[sc_2051_N2_luv1,[604,605,606],[pop_o_matrix_luv1,emp_o_matrix_luv1,senior_o_matrix_luv1,
                                                low_inc_o_matrix_luv1,youth_o_matrix_luv1,pop_d_matrix_luv1,
                                                emp_d_matrix_luv1,low_income_zones_luv1]],
        "2051_N3_APTG":[sc_2051_N3_aptg,[701,702,703],[pop_o_matrix_ogs,emp_o_matrix_ogs,senior_o_matrix_ogs,
                                                low_inc_o_matrix_ogs,youth_o_matrix_ogs,pop_d_matrix_ogs,
                                                emp_d_matrix_ogs,low_income_zones_ogs]],
        "2051_N3_LUV2":[sc_2051_N3_luv2,[704,705,706],[pop_o_matrix_luv2,emp_o_matrix_luv2,senior_o_matrix_luv2,
                                                low_inc_o_matrix_luv2,youth_o_matrix_luv2,pop_d_matrix_luv2,
                                                emp_d_matrix_luv2,low_income_zones_luv2]],
        "2051_N4_APTG":[sc_2051_N4_aptg,[801,802,803],[pop_o_matrix_ogs,emp_o_matrix_ogs,senior_o_matrix_ogs,
                                                low_inc_o_matrix_ogs,youth_o_matrix_ogs,pop_d_matrix_ogs,
                                                emp_d_matrix_ogs,low_income_zones_ogs]],
        "2051_N4_LUV3":[sc_2051_N4_luv3,[804,805,806],[pop_o_matrix_luv3,emp_o_matrix_luv3,senior_o_matrix_luv3,
                                                low_inc_o_matrix_luv3,youth_o_matrix_luv3,pop_d_matrix_luv3,
                                                emp_d_matrix_luv3,low_income_zones_luv3]],
        "2051_N5_APTG":[sc_2051_N5_aptg,[901,902,903],[pop_o_matrix_ogs,emp_o_matrix_ogs,senior_o_matrix_ogs,
                                                low_inc_o_matrix_ogs,youth_o_matrix_ogs,pop_d_matrix_ogs,
                                                emp_d_matrix_ogs,low_income_zones_ogs]],
        "2051_N5_LUV3":[sc_2051_N5_luv3,[904,905,906],[pop_o_matrix_luv3,emp_o_matrix_luv3,senior_o_matrix_luv3,
                                                low_inc_o_matrix_luv3,youth_o_matrix_luv3,pop_d_matrix_luv3,
                                                emp_d_matrix_luv3,low_income_zones_luv3]]} #AM,MD,PM

In [5]:
scens = {"2051_N1_LUV1":[sc_2051_N1_luv1,[504,505,506],[pop_o_matrix_luv1,emp_o_matrix_luv1,senior_o_matrix_luv1,
                                                low_inc_o_matrix_luv1,youth_o_matrix_luv1,pop_d_matrix_luv1,
                                                emp_d_matrix_luv1,low_income_zones_luv1]],
         "2051_N2_LUV1":[sc_2051_N2_luv1,[604,605,606],[pop_o_matrix_luv1,emp_o_matrix_luv1,senior_o_matrix_luv1,
                                                low_inc_o_matrix_luv1,youth_o_matrix_luv1,pop_d_matrix_luv1,
                                                emp_d_matrix_luv1,low_income_zones_luv1]],
         "2051_N3_LUV2":[sc_2051_N3_luv2,[704,705,706],[pop_o_matrix_luv2,emp_o_matrix_luv2,senior_o_matrix_luv2,
                                                low_inc_o_matrix_luv2,youth_o_matrix_luv2,pop_d_matrix_luv2,
                                                emp_d_matrix_luv2,low_income_zones_luv2]],
        "2051_N4_LUV3":[sc_2051_N4_luv3,[804,805,806],[pop_o_matrix_luv3,emp_o_matrix_luv3,senior_o_matrix_luv3,
                                                low_inc_o_matrix_luv3,youth_o_matrix_luv3,pop_d_matrix_luv3,
                                                emp_d_matrix_luv3,low_income_zones_luv3]],
        "2051_N5_LUV3":[sc_2051_N5_luv3,[904,905,906],[pop_o_matrix_luv3,emp_o_matrix_luv3,senior_o_matrix_luv3,
                                                low_inc_o_matrix_luv3,youth_o_matrix_luv3,pop_d_matrix_luv3,
                                                emp_d_matrix_luv3,low_income_zones_luv3]]} #AM,MD,PM

In [6]:
#Matrix naming for inputs

# peak period - 7 hrs
sov_demand_pk = "TRIPS SOV peak"
hov2_demand_pk = "TRIPS HOV2 peak"
hov3_demand_pk = "TRIPS HOV3 peak"
bike_demand_pk = "TRIPS BIKE peak"
walk_demand_pk = "TRIPS WALK peak"
transit_demand_pk = "TRIPS TOTAL TRANSIT peak"

sov_demand_offpk = "TRIPS SOV offpeak"
hov2_demand_offpk = "TRIPS HOV2 offpeak"
hov3_demand_offpk = "TRIPS HOV3 offpeak"
bike_demand_offpk = "TRIPS BIKE offpeak"
walk_demand_offpk = "TRIPS WALK offpeak"
transit_demand_offpk = "TRIPS TOTAL TRANSIT offpeak"

#Truck matrices
truck_l_demand_am = "am_light_2051_20191227"
truck_m_demand_am = "am_medium_2051_20191227"
truck_h_demand_am = "am_heavy_2051_20191227"
truck_l_demand_md = "md_light_2051_20191227"
truck_m_demand_md = "md_medium_2051_20191227"
truck_h_demand_md = "md_heavy_2051_20191227"

# distance
sov_dist_am = "skim.am_peak.auto.sov.dist"
sov_dist_md = "skim.midday.auto.sov.dist"

#Below for road pricing run
# sov_dist_am = "skim.am_peak.auto.sovt.dist"
# sov_dist_md = "skim.midday.auto.sovt.dist"

# travel time
active_time = "skim.static.bike.time"

sov_time_am = "skim.am_peak.auto.sov.time"
hov2_time_am = "skim.am_peak.auto.hov2.time"
hov3_time_am = "skim.am_peak.auto.hov3.time"
sov_time_md = "skim.midday.auto.sov.time"
hov2_time_md = "skim.midday.auto.hov2.time"
hov3_time_md = "skim.midday.auto.hov3.time"

#Below for the road pricing run
# sov_time_am = "skim.am_peak.auto.sovt.time"
# hov2_time_am = "skim.am_peak.auto.hov2t.time"
# hov3_time_am = "skim.am_peak.auto.hov3t.time"
# sov_time_md = "skim.midday.auto.sovt.time"
# hov2_time_md = "skim.midday.auto.hov2t.time"
# hov3_time_md = "skim.midday.auto.hov3t.time"

#costs
sov_toll_am = "skim.am_peak.auto.sovt.toll_cost"
hov2_toll_am = "skim.am_peak.auto.hov2t.toll_cost"
hov3_toll_am = "skim.am_peak.auto.hov3t.toll_cost"

In [7]:
zones_array = ["all",zones_pd1,shed_central,shed_east,shed_midwest,shed_southwest,shed_west,shed_north,shed_niagara]
rural_array = [shed_rural,zones_pd1,rural_central,rural_east,rural_midwest,rural_southwest,rural_west,rural_north,rural_niagara]
zones_array_2 = [zones_pd1,shed_central,shed_east,shed_midwest,shed_southwest,shed_west,shed_north,shed_niagara]

In [8]:
def MOE_fraction_min(out_file,num1,den1,num2,den2,zones_array):
    
    for origins in zones_array:
        spec = get_mc_spec(None, den1,"+","+",origins,"all",None)
        bottom1 = matrix_calculator(spec)['result']
        spec = get_mc_spec(None, num1,"+","+",origins,"all",None)
        top1 = float(matrix_calculator(spec)['result'])
        
        spec = get_mc_spec(None, den2,"+","+",origins,"all",None)
        bottom2 = matrix_calculator(spec)['result']
        spec = get_mc_spec(None, num2,"+","+",origins,"all",None)
        top2 = float(matrix_calculator(spec)['result'])
        out_file.write("{0:.4f},".format(min(top1/bottom1,top2/bottom2)))
    out_file.write("\n")
        
def MOE_fraction_max(out_file,num1,den1,num2,den2,zones_array):
    
    for origins in zones_array:
        spec = get_mc_spec(None, den1,"+","+",origins,"all",None)
        bottom1 = matrix_calculator(spec)['result']
        spec = get_mc_spec(None, num1,"+","+",origins,"all",None)
        top1 = float(matrix_calculator(spec)['result'])
        
        spec = get_mc_spec(None, den2,"+","+",origins,"all",None)
        bottom2 = matrix_calculator(spec)['result']
        spec = get_mc_spec(None, num2,"+","+",origins,"all",None)
        top2 = float(matrix_calculator(spec)['result'])
        out_file.write("{0:.4f},".format(max(top1/bottom1,top2/bottom2)))
    out_file.write("\n")
    
def MOE_fraction_avg(out_file,num1,den1,num2,den2,zones_array):
    
    for origins in zones_array:
        spec = get_mc_spec(None, den1,"+","+",origins,"all",None)
        bottom1 = matrix_calculator(spec)['result']
        spec = get_mc_spec(None, num1,"+","+",origins,"all",None)
        top1 = float(matrix_calculator(spec)['result'])
        
        spec = get_mc_spec(None, den2,"+","+",origins,"all",None)
        bottom2 = matrix_calculator(spec)['result']
        spec = get_mc_spec(None, num2,"+","+",origins,"all",None)
        top2 = float(matrix_calculator(spec)['result'])
        out_file.write("{0:.4f},".format(((top1/bottom1)+(top2/bottom2))/2))
    out_file.write("\n")
        
truck_generators_gateways = "4182;6019;gt2;gt3;gt4;gt5" # only pearson and hamilton for airports

borders = "gt4"

truck_generators = "4182;6019;gt2;gt3"# only pearson and hamilton for airports

airports = "4182;6019"

In [10]:
out_file = r"C:\Users\PECHEN\Desktop\MOE\PseudoBCR\20210107_BAU_BCR_Metrics.csv"
moef = open(out_file, 'w')
moef.write("TP,")

metrics = ["tran_pht","au_pht","trk_vht","au_vht","tran_pkt","au_pkt","trk_vkt","au_vkt","tran_vkt"]

for origins in zones_array:
    for metric in metrics:
        moef.write(origins + "_" + metric + ",")
moef.write("\n")

for scen in scens.keys():
    print("Now processing {}".format(scen))
    scenario_folder = scens[scen][0]
    matrix_folder = r"\\tore-infs01\PWExternal\451010\281829_GGH\GGHM Runs" + "\\" + scenario_folder
    sc_num_am = scens[scen][1][0]
    sc_num_md = scens[scen][1][1]
    #scenarios
    sc_am = eb.scenario(sc_num_am)
    sc_md = eb.scenario(sc_num_md)
    #networks
    ntwk_am = sc_am.get_network()
    ntwk_md = sc_md.get_network()  
    #Shed VKT Section

    ntwks = [ntwk_am,ntwk_md]
    
    matrix_list = {transit_demand_pk:"",
                  transit_demand_offpk:""}
    
    clear_matrices(eb,[hdr_bau_peak, hdr_bau_offpeak,local_fare_mat],"FULL")
    
    for matrix_name in matrix_list.keys():
        matrix_list[matrix_name] = eb.available_matrix_identifier('FULL')
        print matrix_name
        create_matrix(matrix_id = matrix_list[matrix_name],
                      matrix_name= matrix_name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_").strip()[:40],
                      matrix_description = matrix_name)

    import_matrices_from_directory(matrix_folder, matrix_list, ".mdf",sc_am) 

    zones_vkt = [{"all":{"vkt": 0.0, "truck_vkt": 0.0,"pkt": 0.0, "vht": 0.0,"truck_vht":0.0, "pht": 0.0},
                 zones_pd1:{"vkt": 0.0, "truck_vkt": 0.0,"pkt": 0.0, "vht": 0.0,"truck_vht":0.0, "pht": 0.0},
                 shed_central:{"vkt": 0.0, "truck_vkt": 0.0,"pkt": 0.0, "vht": 0.0,"truck_vht":0.0, "pht": 0.0},
                 shed_east:{"vkt": 0.0, "truck_vkt": 0.0,"pkt": 0.0, "vht": 0.0,"truck_vht":0.0, "pht": 0.0},
                 shed_midwest:{"vkt": 0.0, "truck_vkt": 0.0,"pkt": 0.0, "vht": 0.0,"truck_vht":0.0, "pht": 0.0},
                 shed_southwest:{"vkt": 0.0, "truck_vkt": 0.0,"pkt": 0.0, "vht": 0.0,"truck_vht":0.0, "pht": 0.0},
                 shed_west:{"vkt": 0.0, "truck_vkt": 0.0,"pkt": 0.0, "vht": 0.0,"truck_vht":0.0, "pht": 0.0},
                 shed_north:{"vkt": 0.0, "truck_vkt": 0.0,"pkt": 0.0, "vht": 0.0,"truck_vht":0.0, "pht": 0.0},
                 shed_niagara:{"vkt": 0.0, "truck_vkt": 0.0,"pkt": 0.0, "vht": 0.0,"truck_vht":0.0, "pht": 0.0}},
                {"all":{"vkt": 0.0, "truck_vkt": 0.0,"pkt": 0.0, "vht": 0.0,"truck_vht":0.0, "pht": 0.0},
                 zones_pd1:{"vkt": 0.0, "truck_vkt": 0.0,"pkt": 0.0, "vht": 0.0,"truck_vht":0.0, "pht": 0.0},
                 shed_central:{"vkt": 0.0, "truck_vkt": 0.0,"pkt": 0.0, "vht": 0.0,"truck_vht":0.0, "pht": 0.0},
                 shed_east:{"vkt": 0.0, "truck_vkt": 0.0,"pkt": 0.0, "vht": 0.0,"truck_vht":0.0, "pht": 0.0},
                 shed_midwest:{"vkt": 0.0, "truck_vkt": 0.0,"pkt": 0.0, "vht": 0.0,"truck_vht":0.0, "pht": 0.0},
                 shed_southwest:{"vkt": 0.0, "truck_vkt": 0.0,"pkt": 0.0, "vht": 0.0,"truck_vht":0.0, "pht": 0.0},
                 shed_west:{"vkt": 0.0, "truck_vkt": 0.0,"pkt": 0.0, "vht": 0.0,"truck_vht":0.0, "pht": 0.0},
                 shed_north:{"vkt": 0.0, "truck_vkt": 0.0,"pkt": 0.0, "vht": 0.0,"truck_vht":0.0, "pht": 0.0},
                 shed_niagara:{"vkt": 0.0, "truck_vkt": 0.0,"pkt": 0.0, "vht": 0.0,"truck_vht":0.0, "pht": 0.0}}]

    truck_vols = ["@volau_htruck","@volau_mtruck","@volau_ltruck"]
    auto_vols = ["@volau1_nt","@volau1_toll",\
                 "@volau2_nt_nh","@volau2_nt_hov","@volau2_toll_nh","@volau2_toll_hov",\
                "@volau3_nt_nh","@volau3_nt_hov","@volau3_toll_nh","@volau3_toll_hov"]
    
    sov_vols = ["@volau1_nt","@volau1_toll"]
    hov2_vols = ["@volau2_nt_nh","@volau2_nt_hov","@volau2_toll_nh","@volau2_toll_hov"]
    hov3_vols = ["@volau3_nt_nh","@volau3_nt_hov","@volau3_toll_nh","@volau3_toll_hov"]

    for i in range(2):
        for link in ntwks[i].links():
            add = False 
            #This section determines what shed the link belongs to
            if ((link.type > 860 and link.type < 900) or (link.type >= 325 and link.type <= 328) or link.type == 330 or link.type == 332): #North
                shed = shed_north
                add = True
            elif ((link.type > 101 and link.type <200) or link.type == 100 or link.type ==333 or link.type ==329 or link.type ==331): #Central
                shed = shed_central
                add = True
            elif link.type == 101: #PD1
                shed = zones_pd1
                add = True
            elif ((link.type >199 and link.type <300) or (link.type > 899 and link.type < 950)): # East
                shed = shed_east
                add = True
            elif link.type == 820 or link.type ==850: # West
                shed = shed_west
                add = True
            elif (link.type >599 and link.type<700) or link.type ==810 or link.type == 800: #Southwest
                shed = shed_southwest
                add = True
            elif link.type == 700: # Niagara
                shed = shed_niagara
                add = True
            elif link.type >= 400 and link.type < 600: # Midwest
                shed = shed_midwest
                add = True
            if add:
                if link.volume_delay_func > 0 and link.volume_delay_func < 90 and ntwks[i].mode("I") in link.modes:
                    volume = 0.0
                    person_vol =0.0
                    truck_volume = 0.0
                    
                    for vol in sov_vols:
                        volume += link[vol]
                        person_vol += link[vol]*sov_oc
                    for vol in hov2_vols:
                        volume += link[vol]
                        person_vol += link[vol]*hov2_oc
                    for vol in hov3_vols:
                        volume += link[vol]
                        person_vol += link[vol]*hov3_oc
                        
                    for vol in truck_vols:
                        truck_volume += link[vol]

                    zones_vkt[i]["all"]["vkt"] += link.length*volume
                    zones_vkt[i]["all"]["truck_vkt"] += link.length*truck_volume
                    zones_vkt[i][shed]["vkt"] += link.length*volume
                    zones_vkt[i][shed]["truck_vkt"] += link.length*truck_volume
                    
                    zones_vkt[i]["all"]["pkt"] += link.length*person_vol
                    zones_vkt[i][shed]["pkt"] += link.length*person_vol
                    
                    zones_vkt[i]["all"]["vht"] += link.auto_time/60*volume
                    zones_vkt[i]["all"]["truck_vht"] += link.auto_time/60*truck_volume
                    zones_vkt[i][shed]["vht"] += link.auto_time/60*volume
                    zones_vkt[i][shed]["truck_vht"] += link.auto_time/60*truck_volume
                    
                    zones_vkt[i]["all"]["pht"] += link.auto_time/60*person_vol
                    zones_vkt[i][shed]["pht"] += link.auto_time/60*person_vol


    ntwks_tr = [ntwk_am,ntwk_md]

    zones_pkt = [{"all":{"pkt": 0.0,"vkt": 0.0, "pht":0.0},
                 zones_pd1:{"pkt": 0.0,"vkt": 0.0, "pht":0.0},
                 shed_central:{"pkt": 0.0,"vkt": 0.0, "pht":0.0},
                 shed_east:{"pkt": 0.0,"vkt": 0.0, "pht":0.0},
                 shed_midwest:{"pkt": 0.0,"vkt": 0.0, "pht":0.0},
                 shed_southwest:{"pkt": 0.0,"vkt": 0.0, "pht":0.0},
                 shed_west:{"pkt": 0.0,"vkt": 0.0, "pht":0.0},
                 shed_north:{"pkt": 0.0,"vkt": 0.0, "pht":0.0},
                 shed_niagara:{"pkt": 0.0,"vkt": 0.0, "pht":0.0}},
                {"all":{"pkt": 0.0,"vkt": 0.0, "pht":0.0},
                 zones_pd1:{"pkt": 0.0,"vkt": 0.0, "pht":0.0},
                 shed_central:{"pkt": 0.0,"vkt": 0.0, "pht":0.0},
                 shed_east:{"pkt": 0.0,"vkt": 0.0, "pht":0.0},
                 shed_midwest:{"pkt": 0.0,"vkt": 0.0, "pht":0.0},
                 shed_southwest:{"pkt": 0.0,"vkt": 0.0, "pht":0.0},
                 shed_west:{"pkt": 0.0,"vkt": 0.0, "pht":0.0},
                 shed_north:{"pkt": 0.0,"vkt": 0.0, "pht":0.0},
                 shed_niagara:{"pkt": 0.0,"vkt": 0.0, "pht":0.0}}]

    for i in range(2):
        for line in ntwks_tr[i].transit_lines():
            for segment in line.segments():
                link = segment.link
                add = False 
                #This section determines what shed the link belongs to
                if ((link.type > 860 and link.type < 900) or (link.type >= 325 and link.type <= 328) or link.type == 330 or link.type == 332): #North
                    shed = shed_north
                    add = True
                elif ((link.type > 101 and link.type <200) or link.type == 100 or link.type ==333 or link.type ==329 or link.type ==331): #Central
                    shed = shed_central
                    add = True
                elif link.type == 101: #PD1
                    shed = zones_pd1
                    add = True
                elif ((link.type >199 and link.type <300) or (link.type > 899 and link.type < 950)): # East
                    shed = shed_east
                    add = True
                elif link.type == 820 or link.type ==850: # West
                    shed = shed_west
                    add = True
                elif (link.type >599 and link.type<700) or link.type ==810 or link.type == 800: #Southwest
                    shed = shed_southwest
                    add = True
                elif link.type == 700: # Niagara
                    shed = shed_niagara
                    add = True
                elif link.type >= 400 and link.type < 600: # Midwest
                    shed = shed_midwest
                    add = True   
                if add and segment.transit_time > 0 and segment.transit_volume > 0:
                    
                    hdw = line.headway
                    cap = line.vehicle.total_capacity*120/hdw
                    
                    zones_pkt[i]["all"]["pkt"] += link.length * segment.transit_volume
                    zones_pkt[i][shed]["pkt"] += link.length * segment.transit_volume
                    
                    zones_pkt[i]["all"]["pht"] += segment.transit_time/60*segment.transit_volume
                    zones_pkt[i][shed]["pht"] += segment.transit_time/60*segment.transit_volume
                    
                    zones_pkt[i]["all"]["vkt"] += 120/hdw*link.length
                    zones_pkt[i][shed]["vkt"] += 120/hdw*link.length

    moef.write("{},".format("AM"))
    for origins in zones_array:
        moef.write("{0:.4f},".format(zones_pkt[0][origins]["pht"]))
        moef.write("{0:.4f},".format(zones_vkt[0][origins]["pht"]))
        moef.write("{0:.4f},".format(zones_vkt[0][origins]["truck_vht"]))
        moef.write("{0:.4f},".format(zones_vkt[0][origins]["vht"]))
        moef.write("{0:.4f},".format(zones_pkt[0][origins]["pkt"]))
        moef.write("{0:.4f},".format(zones_vkt[0][origins]["pkt"]))
        moef.write("{0:.4f},".format(zones_vkt[0][origins]["truck_vkt"]))
        moef.write("{0:.4f},".format(zones_vkt[0][origins]["vkt"]))
        moef.write("{0:.4f},".format(zones_pkt[0][origins]["vkt"]))
    moef.write("\n")
    
    moef.write("{},".format("MD"))
    for origins in zones_array:
        moef.write("{0:.4f},".format(zones_pkt[1][origins]["pht"]))
        moef.write("{0:.4f},".format(zones_vkt[1][origins]["pht"]))
        moef.write("{0:.4f},".format(zones_vkt[1][origins]["truck_vht"]))
        moef.write("{0:.4f},".format(zones_vkt[1][origins]["vht"]))
        moef.write("{0:.4f},".format(zones_pkt[1][origins]["pkt"]))
        moef.write("{0:.4f},".format(zones_vkt[1][origins]["pkt"]))
        moef.write("{0:.4f},".format(zones_vkt[1][origins]["truck_vkt"]))
        moef.write("{0:.4f},".format(zones_vkt[1][origins]["vkt"]))
        moef.write("{0:.4f},".format(zones_pkt[1][origins]["vkt"]))
    moef.write("\n")
    print("Finished Processing {}".format(scen))
moef.close()

Now processing 2051_N2_LUV1
TRIPS TOTAL TRANSIT peak
TRIPS TOTAL TRANSIT offpeak
Imported matrix mf2
Imported matrix mf1
Finished Importing Matrices
Finished Processing 2051_N2_LUV1
Now processing 2051_N3_LUV2
TRIPS TOTAL TRANSIT peak
TRIPS TOTAL TRANSIT offpeak
Imported matrix mf2
Imported matrix mf1
Finished Importing Matrices
Finished Processing 2051_N3_LUV2
Now processing 2051_N5_LUV3
TRIPS TOTAL TRANSIT peak
TRIPS TOTAL TRANSIT offpeak
Imported matrix mf2
Imported matrix mf1
Finished Importing Matrices
Finished Processing 2051_N5_LUV3
Now processing 2051_N4_LUV3
TRIPS TOTAL TRANSIT peak
TRIPS TOTAL TRANSIT offpeak
Imported matrix mf2
Imported matrix mf1
Finished Importing Matrices
Finished Processing 2051_N4_LUV3
Now processing 2051_N1_LUV1
TRIPS TOTAL TRANSIT peak
TRIPS TOTAL TRANSIT offpeak
Imported matrix mf2
Imported matrix mf1
Finished Importing Matrices
Finished Processing 2051_N1_LUV1


In [13]:
veh_oc = 0.71
safety_oc = 0.11
ghg_val = 50
trk_oc = 1.2043
person_vot = 16.47
truck_vot = 81.73

am_pk_hr_conv = 0.415
md_pk_hr_conv = 0.2

# daily_conv = 1.714 #AM and MD Combined
daily_conv = 2.595 # AM Only
annual_fac = 300
analysis_period = 30


In [17]:
#Using only AM Assignment Results

out_file = r"C:\Users\PECHEN\Desktop\MOE\PseudoBCR\20210107_SL_PseudoBCR_APTG_Networks.csv"
moef = open(out_file, 'w')
moef.write("Scen,MOE,MOE_ID," + ",".join(zones_array) + "\n")

bau_bcr_df = pd.read_csv(r"C:\Users\PECHEN\Desktop\MOE\PseudoBCR\20210107_BAU_BCR_Metrics.csv")

for scen in scens.keys():
    print("Now processing {}".format(scen))
    scenario_folder = scens[scen][0]
    matrix_folder = r"\\tore-infs01\PWExternal\451010\281829_GGH\GGHM Runs" + "\\" + scenario_folder
    sc_num_am = scens[scen][1][0]
    sc_num_md = scens[scen][1][1]
    #scenarios
    sc_am = eb.scenario(sc_num_am)
    sc_md = eb.scenario(sc_num_md)
    #networks
    ntwk_am = sc_am.get_network()
    ntwk_md = sc_md.get_network()
    
    matrix_list = {transit_demand_pk:"",
                  transit_demand_offpk:""}
    
    clear_matrices(eb,[hdr_bau_peak, hdr_bau_offpeak,local_fare_mat],"FULL")
    
    for matrix_name in matrix_list.keys():
        matrix_list[matrix_name] = eb.available_matrix_identifier('FULL')
        print matrix_name
        create_matrix(matrix_id = matrix_list[matrix_name],
                      matrix_name= matrix_name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_").strip()[:40],
                      matrix_description = matrix_name)
        
    import_matrices_from_directory(matrix_folder, matrix_list, ".mdf",sc_am)
    
    #Shed VKT Section

    ntwks = [ntwk_am,ntwk_md]

    zones_vkt = [{"all":{"vkt": 0.0, "truck_vkt": 0.0,"pkt": 0.0, "vht": 0.0,"truck_vht":0.0, "pht": 0.0,
                         "cost":0.0, "bridges": 0.0,"benefits":0.0, "ben_transpht":0.0, "ben_autovkt":0.0,
                         "ben_truckvkt":0.0, "ben_autopht":0.0,"ben_truckvht":0.0},
                 zones_pd1:{"vkt": 0.0, "truck_vkt": 0.0,"pkt": 0.0, "vht": 0.0,"truck_vht":0.0, "pht": 0.0,
                         "cost":0.0, "bridges": 0.0,"benefits":0.0, "ben_transpht":0.0, "ben_autovkt":0.0,
                         "ben_truckvkt":0.0, "ben_autopht":0.0,"ben_truckvht":0.0},
                 shed_central:{"vkt": 0.0, "truck_vkt": 0.0,"pkt": 0.0, "vht": 0.0,"truck_vht":0.0, "pht": 0.0,
                         "cost":0.0, "bridges": 0.0,"benefits":0.0, "ben_transpht":0.0, "ben_autovkt":0.0,
                         "ben_truckvkt":0.0, "ben_autopht":0.0,"ben_truckvht":0.0},
                 shed_east:{"vkt": 0.0, "truck_vkt": 0.0,"pkt": 0.0, "vht": 0.0,"truck_vht":0.0, "pht": 0.0,
                         "cost":0.0, "bridges": 0.0,"benefits":0.0, "ben_transpht":0.0, "ben_autovkt":0.0,
                         "ben_truckvkt":0.0, "ben_autopht":0.0,"ben_truckvht":0.0},
                 shed_midwest:{"vkt": 0.0, "truck_vkt": 0.0,"pkt": 0.0, "vht": 0.0,"truck_vht":0.0, "pht": 0.0,
                         "cost":0.0, "bridges": 0.0,"benefits":0.0, "ben_transpht":0.0, "ben_autovkt":0.0,
                         "ben_truckvkt":0.0, "ben_autopht":0.0,"ben_truckvht":0.0},
                 shed_southwest:{"vkt": 0.0, "truck_vkt": 0.0,"pkt": 0.0, "vht": 0.0,"truck_vht":0.0, "pht": 0.0,
                         "cost":0.0, "bridges": 0.0,"benefits":0.0, "ben_transpht":0.0, "ben_autovkt":0.0,
                         "ben_truckvkt":0.0, "ben_autopht":0.0,"ben_truckvht":0.0},
                 shed_west:{"vkt": 0.0, "truck_vkt": 0.0,"pkt": 0.0, "vht": 0.0,"truck_vht":0.0, "pht": 0.0,
                         "cost":0.0, "bridges": 0.0,"benefits":0.0, "ben_transpht":0.0, "ben_autovkt":0.0,
                         "ben_truckvkt":0.0, "ben_autopht":0.0,"ben_truckvht":0.0},
                 shed_north:{"vkt": 0.0, "truck_vkt": 0.0,"pkt": 0.0, "vht": 0.0,"truck_vht":0.0, "pht": 0.0,
                         "cost":0.0, "bridges": 0.0,"benefits":0.0, "ben_transpht":0.0, "ben_autovkt":0.0,
                         "ben_truckvkt":0.0, "ben_autopht":0.0,"ben_truckvht":0.0},
                 shed_niagara:{"vkt": 0.0, "truck_vkt": 0.0,"pkt": 0.0, "vht": 0.0,"truck_vht":0.0, "pht": 0.0,
                         "cost":0.0, "bridges": 0.0,"benefits":0.0, "ben_transpht":0.0, "ben_autovkt":0.0,
                         "ben_truckvkt":0.0, "ben_autopht":0.0,"ben_truckvht":0.0}},
                {"all":{"vkt": 0.0, "truck_vkt": 0.0,"pkt": 0.0, "vht": 0.0,"truck_vht":0.0, "pht": 0.0,
                         "benefits":0.0},
                 zones_pd1:{"vkt": 0.0, "truck_vkt": 0.0,"pkt": 0.0, "vht": 0.0,"truck_vht":0.0, "pht": 0.0,
                         "benefits":0.0},
                 shed_central:{"vkt": 0.0, "truck_vkt": 0.0,"pkt": 0.0, "vht": 0.0,"truck_vht":0.0, "pht": 0.0,
                         "benefits":0.0},
                 shed_east:{"vkt": 0.0, "truck_vkt": 0.0,"pkt": 0.0, "vht": 0.0,"truck_vht":0.0, "pht": 0.0,
                         "benefits":0.0},
                 shed_midwest:{"vkt": 0.0, "truck_vkt": 0.0,"pkt": 0.0, "vht": 0.0,"truck_vht":0.0, "pht": 0.0,
                         "benefits":0.0},
                 shed_southwest:{"vkt": 0.0, "truck_vkt": 0.0,"pkt": 0.0, "vht": 0.0,"truck_vht":0.0, "pht": 0.0,
                        "benefits":0.0},
                 shed_west:{"vkt": 0.0, "truck_vkt": 0.0,"pkt": 0.0, "vht": 0.0,"truck_vht":0.0, "pht": 0.0,
                         "benefits":0.0},
                 shed_north:{"vkt": 0.0, "truck_vkt": 0.0,"pkt": 0.0, "vht": 0.0,"truck_vht":0.0, "pht": 0.0,
                         "benefits":0.0},
                 shed_niagara:{"vkt": 0.0, "truck_vkt": 0.0,"pkt": 0.0, "vht": 0.0,"truck_vht":0.0, "pht": 0.0,
                        "benefits":0.0}}]

    truck_vols = ["@volau_htruck","@volau_mtruck","@volau_ltruck"]
    auto_vols = ["@volau1_nt","@volau1_toll",\
                 "@volau2_nt_nh","@volau2_nt_hov","@volau2_toll_nh","@volau2_toll_hov",\
                "@volau3_nt_nh","@volau3_nt_hov","@volau3_toll_nh","@volau3_toll_hov"]
    
    sov_vols = ["@volau1_nt","@volau1_toll"]
    hov2_vols = ["@volau2_nt_nh","@volau2_nt_hov","@volau2_toll_nh","@volau2_toll_hov"]
    hov3_vols = ["@volau3_nt_nh","@volau3_nt_hov","@volau3_toll_nh","@volau3_toll_hov"]

    for i in range(2):
        for link in ntwks[i].links():
            add = False 
            #This section determines what shed the link belongs to
            if ((link.type > 860 and link.type < 900) or (link.type >= 325 and link.type <= 328) or link.type == 330 or link.type == 332): #North
                shed = shed_north
                add = True
            elif ((link.type > 101 and link.type <200) or link.type == 100 or link.type ==333 or link.type ==329 or link.type ==331): #Central
                shed = shed_central
                add = True
            elif link.type == 101: #PD1
                shed = zones_pd1
                add = True
            elif ((link.type >199 and link.type <300) or (link.type > 899 and link.type < 950)): # East
                shed = shed_east
                add = True
            elif link.type == 820 or link.type ==850: # West
                shed = shed_west
                add = True
            elif (link.type >599 and link.type<700) or link.type ==810 or link.type == 800: #Southwest
                shed = shed_southwest
                add = True
            elif link.type == 700: # Niagara
                shed = shed_niagara
                add = True
            elif link.type >= 400 and link.type < 600: # Midwest
                shed = shed_midwest
                add = True
            if add:
                if i == 0: #Only need to do cost for AM network
                    if link["@rcost"] > 0:
                        r_imp = link["@rcost"]
                        if shed == shed_central or shed == zones_pd1 or shed == shed_midwest or shed == shed_southwest:
                            zones_vkt[i]["all"]["cost"] += roadCosts[r_imp][0]*link.length
                            zones_vkt[i][shed]["cost"] += roadCosts[r_imp][0]*link.length
                        else:
                            zones_vkt[i]["all"]["cost"] += roadCosts[r_imp][1]*link.length
                            zones_vkt[i][shed]["cost"] += roadCosts[r_imp][1]*link.length
                    if link["@tcost"] > 0:
                        t_imp = link["@tcost"]
                        zones_vkt[i]["all"]["cost"] += transitCosts[t_imp]*link.length
                        zones_vkt[i][shed]["cost"] += transitCosts[t_imp]*link.length
                            
                    if link["@bcost"] > 0:
                        b_imp = link["@bcost"]
                        if b_imp == 1 or b_imp == 8:
                            zones_vkt[i]["all"]["bridges"] += 1
                            zones_vkt[i][shed]["bridges"] += 1
                        if b_imp == 4: # GO Station case
                            if (link.type > 101 and link.type <200):
                                zones_vkt[i]["all"]["cost"] += bridgeCost[b_imp][0]
                                zones_vkt[i][shed]["cost"] += bridgeCost[b_imp][0]
                            else:
                                zones_vkt[i]["all"]["cost"] += bridgeCost[b_imp][1]
                                zones_vkt[i][shed]["cost"] += bridgeCost[b_imp][1]
                        else:
                            if shed == shed_central or shed == zones_pd1 or shed == shed_midwest or shed == shed_southwest:
                                zones_vkt[i]["all"]["cost"] += bridgeCost[b_imp][0]
                                zones_vkt[i][shed]["cost"] += bridgeCost[b_imp][0]
                            else:
                                zones_vkt[i]["all"]["cost"] += bridgeCost[b_imp][1]
                                zones_vkt[i][shed]["cost"] += bridgeCost[b_imp][1]
                if link.volume_delay_func > 0 and link.volume_delay_func < 90 and ntwks[i].mode("I") in link.modes:
                    volume = 0.0
                    person_vol =0.0
                    truck_volume = 0.0
                    
                    for vol in sov_vols:
                        volume += link[vol]
                        person_vol += link[vol]*sov_oc
                    for vol in hov2_vols:
                        volume += link[vol]
                        person_vol += link[vol]*hov2_oc
                    for vol in hov3_vols:
                        volume += link[vol]
                        person_vol += link[vol]*hov3_oc
                        
                    for vol in truck_vols:
                        truck_volume += link[vol]

                    zones_vkt[i]["all"]["vkt"] += link.length*volume
                    zones_vkt[i]["all"]["truck_vkt"] += link.length*truck_volume
                    zones_vkt[i][shed]["vkt"] += link.length*volume
                    zones_vkt[i][shed]["truck_vkt"] += link.length*truck_volume
                    
                    zones_vkt[i]["all"]["pkt"] += link.length*person_vol
                    zones_vkt[i][shed]["pkt"] += link.length*person_vol
                    
                    zones_vkt[i]["all"]["vht"] += link.auto_time/60*volume
                    zones_vkt[i]["all"]["truck_vht"] += link.auto_time/60*truck_volume
                    zones_vkt[i][shed]["vht"] += link.auto_time/60*volume
                    zones_vkt[i][shed]["truck_vht"] += link.auto_time/60*truck_volume
                    
                    zones_vkt[i]["all"]["pht"] += link.auto_time/60*person_vol
                    zones_vkt[i][shed]["pht"] += link.auto_time/60*person_vol


    ntwks_tr = [ntwk_am,ntwk_md]

    zones_pkt = [{"all":{"pkt": 0.0,"vkt": 0.0, "pht":0.0},
                 zones_pd1:{"pkt": 0.0,"vkt": 0.0, "pht":0.0},
                 shed_central:{"pkt": 0.0,"vkt": 0.0, "pht":0.0},
                 shed_east:{"pkt": 0.0,"vkt": 0.0, "pht":0.0},
                 shed_midwest:{"pkt": 0.0,"vkt": 0.0, "pht":0.0},
                 shed_southwest:{"pkt": 0.0,"vkt": 0.0, "pht":0.0},
                 shed_west:{"pkt": 0.0,"vkt": 0.0, "pht":0.0},
                 shed_north:{"pkt": 0.0,"vkt": 0.0, "pht":0.0},
                 shed_niagara:{"pkt": 0.0,"vkt": 0.0, "pht":0.0}},
                {"all":{"pkt": 0.0,"vkt": 0.0, "pht":0.0},
                 zones_pd1:{"pkt": 0.0,"vkt": 0.0, "pht":0.0},
                 shed_central:{"pkt": 0.0,"vkt": 0.0, "pht":0.0},
                 shed_east:{"pkt": 0.0,"vkt": 0.0, "pht":0.0},
                 shed_midwest:{"pkt": 0.0,"vkt": 0.0, "pht":0.0},
                 shed_southwest:{"pkt": 0.0,"vkt": 0.0, "pht":0.0},
                 shed_west:{"pkt": 0.0,"vkt": 0.0, "pht":0.0},
                 shed_north:{"pkt": 0.0,"vkt": 0.0, "pht":0.0},
                 shed_niagara:{"pkt": 0.0,"vkt": 0.0, "pht":0.0}}]

    for i in range(2):
        for line in ntwks_tr[i].transit_lines():
            for segment in line.segments():
                link = segment.link
                add = False 
                #This section determines what shed the link belongs to
                if ((link.type > 860 and link.type < 900) or (link.type >= 325 and link.type <= 328) or link.type == 330 or link.type == 332): #North
                    shed = shed_north
                    add = True
                elif ((link.type > 101 and link.type <200) or link.type == 100 or link.type ==333 or link.type ==329 or link.type ==331): #Central
                    shed = shed_central
                    add = True
                elif link.type == 101: #PD1
                    shed = zones_pd1
                    add = True
                elif ((link.type >199 and link.type <300) or (link.type > 899 and link.type < 950)): # East
                    shed = shed_east
                    add = True
                elif link.type == 820 or link.type ==850: # West
                    shed = shed_west
                    add = True
                elif (link.type >599 and link.type<700) or link.type ==810 or link.type == 800: #Southwest
                    shed = shed_southwest
                    add = True
                elif link.type == 700: # Niagara
                    shed = shed_niagara
                    add = True
                elif link.type >= 400 and link.type < 600: # Midwest
                    shed = shed_midwest
                    add = True   
                if add and segment.transit_time > 0 and segment.transit_volume > 0:
                    
                    hdw = line.headway
                    cap = line.vehicle.total_capacity*120/hdw
                    
                    zones_pkt[i]["all"]["pkt"] += link.length * segment.transit_volume
                    zones_pkt[i][shed]["pkt"] += link.length * segment.transit_volume
                    
                    zones_pkt[i]["all"]["pht"] += segment.transit_time/60*segment.transit_volume
                    zones_pkt[i][shed]["pht"] += segment.transit_time/60*segment.transit_volume
                    
                    zones_pkt[i]["all"]["vkt"] += 120/hdw*link.length
                    zones_pkt[i][shed]["vkt"] += 120/hdw*link.length
                    
    bau_trips = bau_bcr_df["tr_trips"].sum()
    
    spec = get_mc_spec(None, matrix_list[transit_demand_pk],"+","+","all","all",None)
    pk_trips = matrix_calculator(spec)['result']
    
    spec = get_mc_spec(None, matrix_list[transit_demand_offpk],"+","+","all","all",None)
    offpk_trips = matrix_calculator(spec)['result']
    
    ratio = bau_trips/(pk_trips + offpk_trips)
    
    for zones in zones_array:
        
        trans_pht_am = person_vot*0.61*(bau_bcr_df[bau_bcr_df.TP == "AM"][zones + "_tran_pht"].values[0] - zones_pkt[0][zones]["pht"]*ratio)
#         trans_pht_md = person_vot*(bau_bcr_df[bau_bcr_df.TP == "MD"][zones + "_tran_pht"].values[0] - zones_pkt[1][zones]["pht"]*ratio)
        
        zones_vkt[0][zones]["benefits"] += daily_conv*annual_fac*analysis_period*((trans_pht_am/am_pk_hr_conv))# + (trans_pht_md/md_pk_hr_conv))
        zones_vkt[0][zones]["ben_transpht"] += daily_conv*annual_fac*analysis_period*((trans_pht_am/am_pk_hr_conv))# + (trans_pht_md/md_pk_hr_conv))
        
        auto_vkt_am = (veh_oc)*(bau_bcr_df[bau_bcr_df.TP == "AM"][zones + "_au_vkt"].values[0] - zones_vkt[0][zones]["vkt"])
#         auto_vkt_md = (veh_oc)*(bau_bcr_df[bau_bcr_df.TP == "MD"][zones + "_au_vkt"].values[0] - zones_vkt[1][zones]["vkt"])
        
        zones_vkt[0][zones]["benefits"] += daily_conv*annual_fac*analysis_period*((auto_vkt_am/am_pk_hr_conv))# + (auto_vkt_md/md_pk_hr_conv))
        zones_vkt[0][zones]["ben_autovkt"] += daily_conv*annual_fac*analysis_period*((auto_vkt_am/am_pk_hr_conv))# + (auto_vkt_md/md_pk_hr_conv))
        
        
        truck_vkt_am = trk_oc*(bau_bcr_df[bau_bcr_df.TP == "AM"][zones + "_trk_vkt"].values[0] -zones_vkt[0][zones]["truck_vkt"])
#         truck_vkt_md = trk_oc*(bau_bcr_df[bau_bcr_df.TP == "MD"][zones + "_trk_vkt"].values[0] -zones_vkt[1][zones]["truck_vkt"])

        zones_vkt[0][zones]["benefits"] += daily_conv*annual_fac*analysis_period*((truck_vkt_am/am_pk_hr_conv))# + (truck_vkt_md/md_pk_hr_conv))
        zones_vkt[0][zones]["ben_truckvkt"] += daily_conv*annual_fac*analysis_period*((truck_vkt_am/am_pk_hr_conv))# + (truck_vkt_md/md_pk_hr_conv))
        
        
        truck_vht_am = truck_vot*(bau_bcr_df[bau_bcr_df.TP == "AM"][zones + "_trk_vht"].values[0] -zones_vkt[0][zones]["truck_vht"])
#         truck_vht_md = truck_vot*(bau_bcr_df[bau_bcr_df.TP == "MD"][zones + "_trk_vht"].values[0] -zones_vkt[1][zones]["truck_vht"])
    
        zones_vkt[0][zones]["benefits"] += daily_conv*annual_fac*analysis_period*((truck_vht_am/am_pk_hr_conv))# + (truck_vht_md/md_pk_hr_conv))
        zones_vkt[0][zones]["ben_truckvht"] += daily_conv*annual_fac*analysis_period*((truck_vht_am/am_pk_hr_conv))# + (truck_vht_md/md_pk_hr_conv))
    
    
        auto_pht_am = person_vot*(bau_bcr_df[bau_bcr_df.TP == "AM"][zones + "_au_pht"].values[0] - zones_vkt[0][zones]["pht"])
#         auto_pht_md = person_vot*(bau_bcr_df[bau_bcr_df.TP == "MD"][zones + "_au_pht"].values[0] - zones_vkt[1][zones]["pht"])

        zones_vkt[0][zones]["benefits"] += daily_conv*annual_fac*analysis_period*((auto_pht_am/am_pk_hr_conv))# + (auto_pht_md/md_pk_hr_conv))
        zones_vkt[0][zones]["ben_autopht"] += daily_conv*annual_fac*analysis_period*((auto_pht_am/am_pk_hr_conv))# + (auto_pht_md/md_pk_hr_conv))
        
    moef.write("{},# of new or expanded transportation corridors crossing major watercourse,040203,".format(scen))

    for origins in zones_array:
        moef.write("{0:.4f},".format(zones_vkt[0][origins]["bridges"]))
    moef.write("\n")
    
    moef.write("{},High level cost estimate ,050101,".format(scen))

    for origins in zones_array:
        moef.write("{0:.4f},".format(zones_vkt[0][origins]["cost"]))
    moef.write("\n")
    
    moef.write("{},Benefits - Transit PHT,0501021,".format(scen))

    for origins in zones_array:
        moef.write("{0:.4f},".format(zones_vkt[0][origins]["ben_transpht"]))
    moef.write("\n")
    
    moef.write("{},Benefits - Auto VKT,0501022,".format(scen))

    for origins in zones_array:
        moef.write("{0:.4f},".format(zones_vkt[0][origins]["ben_autovkt"]))
    moef.write("\n")
    
    moef.write("{},Benefits - Truck VKT,0501023,".format(scen))

    for origins in zones_array:
        moef.write("{0:.4f},".format(zones_vkt[0][origins]["ben_truckvkt"]))
    moef.write("\n")
    
    moef.write("{},Benefits - Auto PHT,0501024,".format(scen))

    for origins in zones_array:
        moef.write("{0:.4f},".format(zones_vkt[0][origins]["ben_autopht"]))
    moef.write("\n")
    
    moef.write("{},Benefits - Truck PHT,0501025,".format(scen))

    for origins in zones_array:
        moef.write("{0:.4f},".format(zones_vkt[0][origins]["ben_truckvht"]))
    moef.write("\n")
    
    moef.write("{},Pseudo BCR ,050102,".format(scen))

    for origins in zones_array:
        if zones_vkt[0][origins]["cost"] > 0:
            moef.write("{0:.4f},".format(zones_vkt[0][origins]["benefits"]/zones_vkt[0][origins]["cost"]))
        else:
            moef.write("-,")
    moef.write("\n")
    
    print("Finished Processing {}".format(scen))
moef.close()

In [9]:
out_file = r"C:\Users\bcrane\Documents\GGH - Medium List\MOE\20201026_Cost_BCR_BAU_Trips.csv"
moef = open(out_file, 'w')
moef.write("TP,Trips,\n")

for scen in scens.keys():
    print("Now processing {}".format(scen))
    scenario_folder = scens[scen][0]
    matrix_folder = r"\\tore-infs01\PWExternal\451010\281829_GGH\GGHM Runs" + "\\" + scenario_folder
    
    sc_num_am = scens[scen][1][0]
#     sc_num_md = scens[scen][1][1]
    #scenarios
    sc_am = eb.scenario(sc_num_am)
#     sc_md = eb.scenario(sc_num_md)
    #networks
#     ntwk_am = sc_am.get_network()
#     ntwk_md = sc_md.get_network()
    
    matrix_list = {transit_demand_pk:"",
                  transit_demand_offpk:""}
    
    clear_matrices(eb,[hdr_bau_peak, hdr_bau_offpeak,local_fare_mat],"FULL")
    
    for matrix_name in matrix_list.keys():
        matrix_list[matrix_name] = eb.available_matrix_identifier('FULL')
        print matrix_name
        create_matrix(matrix_id = matrix_list[matrix_name],
                      matrix_name= matrix_name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_").strip()[:40],
                      matrix_description = matrix_name)
        
    import_matrices_from_directory(matrix_folder, matrix_list, ".mdf",sc_am)
    
    spec = get_mc_spec(None, matrix_list[transit_demand_pk],"+","+","all","all",None)
    pk_trips = matrix_calculator(spec)['result']
    
    spec = get_mc_spec(None, matrix_list[transit_demand_offpk],"+","+","all","all",None)
    offpk_trips = matrix_calculator(spec)['result']
    
    moef.write("AM,{0},\nMD,{1},\n".format(pk_trips,offpk_trips))
moef.close()

Now processing 2051_BAU_OGS
TRIPS TOTAL TRANSIT peak
TRIPS TOTAL TRANSIT offpeak
Imported matrix mf2
Imported matrix mf1
Finished Importing Matrices


In [10]:
moef.close()

In [12]:
df

,TP,all_tran_pht,all_au_pht,all_trk_vht,all_au_vht,all_tran_pkt,all_au_pkt,all_trk_vkt,all_au_vkt,all_tran_vkt,...,gs70;gs71_tran_pht,gs70;gs71_au_pht,gs70;gs71_trk_vht,gs70;gs71_au_vht,gs70;gs71_tran_pkt,gs70;gs71_au_pkt,gs70;gs71_trk_vkt,gs70;gs71_au_vkt,gs70;gs71_tran_vkt,Unnamed: 82
0,AM,444703.7545,1200654.1699,154990.7873,850959.0049,12460169.6816,43406069.9232,6995928.8333,30950570.2916,183021.8426,...,3626.8949,32675.1750,5094.9143,23317.1794,106078.7614,1734410.6929,329830.6934,1246010.3933,4944.1635,NaN
1,MD,474375.0044,284872.8002,97331.5214,200025.1270,14014574.6956,17311077.3275,6946988.1301,12292288.1691,123005.7153,...,5935.2690,11859.2722,3450.4983,8306.3541,171236.2166,774474.4194,296245.8666,550403.1474,4046.6367,NaN


In [13]:
df[df.TP == "AM"]["all_tran_pht"].values[0]

444703.75449999998